# For a mab ID if there is a droupout but the same mab ID has a good sequence we dont care about them
- summary of HC or just LC as well.. per plate stats. 
- if no shown on web then a Dec dropout is ok
- up to the first dot can be the same (D4/81.1, D4/81.2)
- dont rely on columns from the duplicates too much for dropout etc. 
- generate all the information as well


# Duplications and subclones
- project name, parent clone number, subclone number (sibilings)
- for every clone for it to have a cooraborating sequence in the database. If .1 has sequence than we want .2,.3 etc to have the same sequence. 
- what number of the duplicates for an ASV have the same parent so are subclones
- 49 clones with 47 parents... is it real? What plates are these from?
    - some other sequences that keeps showing up but doesnt 
- same subclone but one is off.. red - one typo but two 
- which are ok? Which need to be binned or need more analysis, which are close on the plate
- do some have sequences with multiple chains across multiple ASVs. 

- ASV amount
- eventually deactivating them?
- across plates or across wells
- **coding should match undergrad**
- **if on the same plate could be a contaminant... different plate typo?**


# How will this work better in the future
- what happens if a trimmer id is changed
- DONT USE THE TIMMER ID IN THE DROPOUT FILE OR THE TRIMMERID IN THE DUPLICATE FILE BECAUSE WHAT IF THEY CHANGE IN THE METADATA


# Sequence deactivation
- what if we can get rid of certain sequences... so like duplicates with one sketch HC but a chill LC
- 




In [60]:
import pandas as pd
data = pd.read_csv('Duplicated_2021_02_01.csv')
data.head()

,DuplicatedCategory,ASV,DuplicatedIn,MabIDs,ParentIDs,MabCount,ParentCount
0,1,ATCTCCTCACTAGAGCCCCCATCAGAGCATGGCTGTCCTGGTGCTG...,"P21_D11, P21_D12, P21_E1, P21_E3, P21_E4, P21_...","N420/24.1, N420/24.1, N420/24.10, N420/24.10.1...",N420/24,14,1
1,1,AACATGTGTCCAATGTCCTCTCCACAGACACTGAACACACTGACTC...,"P21_D11, P21_D12, P21_E1, P21_E3, P21_E4, P21_...","N420/24.1, N420/24.1, N420/24.10, N420/24.10.1...",N420/24,14,1
2,1,ACAAGTGTGCAGCCATGGGCAGGCTTACTTCTTCATTCCTGCTACT...,"P22_F11, P22_F12, P22_F2, P22_F3, P22_F4, P22_...","N424/48.1, N424/48.1, N424/48.1.1, N424/48.1.1...",N424/48,12,1
3,1,ACAGTCACTGAAAACATTGACTCTAATCATGGAATGTAACTGGATA...,"P2_H10, P5_D10, P5_D11, P5_D12, P5_D5, P5_D6, ...","N86/8.3.5, N86/8.3.5.1, N86/8.3.5.1, N86/8.3.5...",N86/8,11,1
4,1,AAGCGAGTGACCAGTTAGTCTTAAGGCACCACTTCTTAGACATCAT...,"P17_D6, P17_D7, P17_D8, P17_F6, P17_G10, P17_H...","L125/76.1, L125/76.1, L125/76.1, L125/76.2, L1...",L125/76,11,1


In [53]:
dropout = pd.read_csv("../static_data/2021-02-15.tsv", sep='\t')
# focus category 1 first..
dropout.head()

,sample_name,trimmer_id,PlateName,SeqYear,Dropout? Missing,Dropout resolution,Category,ShowOnWeb,ProteinTarget,ChangeDate,ChangeReason
0,P65_H2,10% N48676.1 + 30% L120/12.1 + 60% L71/5.5 cells,plate65,Year1,n/a-n/a,NaN,NaN,F,NaN,20200619.0,Mixed sample
1,P65_H7,10% N48676.1 + 30% L120/12.1 + 60% L71/5.5 RNA,plate65,Year1,n/a-n/a,NaN,NaN,F,NaN,20200619.0,Mixed sample
2,P2_E4,1D8.1,plate2,Year1,n/a-n/a,NaN,1.0,T,LRRK2/Dardarin,20201109.0,"Removed ""Restricted clone"" status"
3,P71_E1,1D8.4,plate71,Year2,n/a-n/a,NaN,3.0,T,LRRK2/Dardarin,20201109.0,NaN
4,P71_E2,1D8.5,plate71,Year2,n/a-n/a,NaN,3.0,T,LRRK2/Dardarin,20201109.0,NaN


    
# Duplicates
## -1: These did not work in the code
- likely due to some having subclone in one but not another (TODO write new rule for this)
- also occurs when no / in the ID
- often positive control ones


## 0 CATEGORY: These are ok. No sublone in them
- check if all entries have no dots.. do not include in subsequent analysis

## 1 CATEGORY: These are ok as well. All IDs match.
- check if all the same

## 2 CATEGORY: just the subclone
- check if all same up to the subclone type (up to the first dot) (projects and parents all the same but cant be all same)


## 3 CATEGORY: likely typos with groups more then 2    
- check if project is the same but parent is different and subclone is the same (indicates typo in parent)
- 3

- check if project is different but parent is same and subclone is the same (indicates typo in project)
- 3.5

- check if project is different and parent is different and subclone is the same (indicates typo in parent and subclone)
- 3.6

## 4 CATEGORY: same as 3 but should only be for 2 entry groups
- check if project is the same but parent is different and subclone is the same (indicates typo in parent)
- 4

- check if project is different but parent is same and subclone is the same (indicates typo in project)
- 4.5

- check if project is different and parent is different and subclone is the same (indicates typo in parent and subclone)
- 4.6


## 5 CATEGORY: bad?

In [61]:
def plate_to_mab(plate_loc):
    try:
        entry = dropout[dropout['sample_name']==plate_loc]
        return(list(entry['trimmer_id'])[0])
    except:
        print("ERROR: there is an issue with %s" %(plate_loc))
plate_to_mab('P28_A8')

'N396/54.1'

In [67]:

mycatlist = []
for index,row in data.iterrows():
    # todo should be the duplicated in and grab the mabID
    #print(row['DuplicatedIn'])
    print(row['DuplicatedIn'])
    meta_listupdate = [plate_to_mab(i) for i in row['DuplicatedIn'].split(',')] 
    print(meta_listupdate)
    mab_id_list = [i.replace(' ','') for i in meta_listupdate]
    #print([i.split('.')[0].replace(' ','') for i in mab_id_list])
    #print(len(set([i.split('.')[0].replace(' ','') for i in mab_id_list])))
    
    try:
        projects = [i.split('/')[0].replace(' ','') for i in mab_id_list]
        parents = [i.split('/')[1].split('.')[0].replace(' ', '') for i in mab_id_list]
        subclone = [i.split('/')[1].split('.')[1].replace(' ', '') for i in mab_id_list]
        #print(projects)
        #print(parents)
        #print(subclone)    

    
        # 0 CATEGORY: These are ok. No sublone in them
        # check if all entries have no dots.. do not include in subsequent analysis
        if sum([1 for i in mab_id_list if '.' in i])<1:
            mycatlist.append(0)    

        # 1 CATEGORY: These are ok as well. All IDs match.
        # check if all the same
        elif len(set(mab_id_list))==1:
            mycatlist.append(1)

        # 2 CATEGORY: just the subclone
        # check if all same up to the subclone type (up to the first dot) (projects and parents all 
        # the same but cant be all same)
        elif len(set(projects))==1 and len(set(parents))==1:
            mycatlist.append(2)


        # 3 CATEGORY: likely typos with groups more then 2    
        # check if project is the same but parent is different and subclone is the same (indicates typo in parent)
        elif len(set(projects))==1 and len(set(parents))==2 and len(set(subclone))==1 and len(mab_id_list)>2:
            mycatlist.append(3)

        # check if project is different but parent is same and subclone is the same (indicates typo in parent)
        elif len(set(projects))==2 and len(set(parents))==1 and len(set(subclone))==1 and len(mab_id_list)>2:
            mycatlist.append(3.5)

        # check if project is different and parent is different and subclone is the same (indicates typo in parent and subclone)
        elif len(set(projects))==1 and len(set(parents))==2 and len(set(subclone))==2 and len(mab_id_list)>2:
            mycatlist.append(3.6)


        # 4 CATEGORY: same as 3 but no requirement for more then 2 entries
        # check if project is the same but parent is different and subclone is the same (indicates typo in parent)
        elif len(set(projects))==1 and len(set(parents))==2 and len(set(subclone))==1:
            mycatlist.append(4)

        # check if project is different but parent is same and subclone is the same (indicates typo in parent)
        elif len(set(projects))==2 and len(set(parents))==1 and len(set(subclone))==1:
            mycatlist.append(4.5)

        # check if project is different and parent is different and subclone is the same (indicates typo in parent and subclone)
        elif len(set(projects))==1 and len(set(parents))==2 and len(set(subclone))==2:
            mycatlist.append(4.6)


        # 5 CATEGORY: bad?
        else:
            mycatlist.append(5)
            
            
    except:
        # 0 CATEGORY: These are ok. No sublone in them
        # check if all entries have no dots.. do not include in subsequence analysis
        if sum([1 for i in mab_id_list if '.' in i])<1:
            mycatlist.append(0)    

        # 1 CATEGORY: These are ok as well. All IDs match.
        # check if all the same
        elif len(set(mab_id_list))==1:
            mycatlist.append(1)
            
        # -1 make my own category for failed ones
        else:
            mycatlist.append(-1)

            
            
        

P21_D11, P21_D12, P21_E1, P21_E3, P21_E4, P21_E5, P21_E7, P21_E8, P21_H2, P21_H3, P21_H4, P21_H5, P3_F2, PS2_F7
ERROR: there is an issue with  P21_D12
ERROR: there is an issue with  P21_E1
ERROR: there is an issue with  P21_E3
ERROR: there is an issue with  P21_E4
ERROR: there is an issue with  P21_E5
ERROR: there is an issue with  P21_E7
ERROR: there is an issue with  P21_E8
ERROR: there is an issue with  P21_H2
ERROR: there is an issue with  P21_H3
ERROR: there is an issue with  P21_H4
ERROR: there is an issue with  P21_H5
ERROR: there is an issue with  P3_F2
ERROR: there is an issue with  PS2_F7
['N420/24.1', None, None, None, None, None, None, None, None, None, None, None, None, None]


AttributeError: 'NoneType' object has no attribute 'replace'

In [24]:
data['mycat'] = mycatlist


In [25]:
for index,row in data.iterrows():
    if row['mycat'] != row['DuplicatedCategory']:
        #if row['mycat'] != 6:
            print([row['mycat'], row['DuplicatedCategory'], row['MabIDs']])
        

[2.0, 1, 'N420/24.1, N420/24.1, N420/24.10, N420/24.10.1, N420/24.10.2, N420/24.10.3, N420/24.10.4, N420/24.10.5, N420/24.2, N420/24.3, N420/24.5, N420/24.6, N420/24.7, N420/24.9']
[2.0, 1, 'N420/24.1, N420/24.1, N420/24.10, N420/24.10.1, N420/24.10.2, N420/24.10.3, N420/24.10.4, N420/24.10.5, N420/24.2, N420/24.3, N420/24.5, N420/24.6, N420/24.7, N420/24.9']
[2.0, 1, 'N424/48.1, N424/48.1, N424/48.1.1, N424/48.1.1, N424/48.1.2, N424/48.1.3, N424/48.1.4, N424/48.1.5, N424/48.2, N424/48.3, N424/48.4, N424/48.5']
[2.0, 1, 'N86/8.3.5, N86/8.3.5.1, N86/8.3.5.1, N86/8.3.5.10, N86/8.3.5.3, N86/8.3.5.5, N86/8.3.5.7, N86/8.3.5.8, N86/8.3.5.9, N86/8.3.7, N86/8.5.3.1']
[2.0, 1, 'L125/76.1, L125/76.1, L125/76.1, L125/76.2, L125/76.2, L125/76.2, L125/76.4, L125/76.5, L125/76.6, L125/76.7, L125/76.8']
[2.0, 1, 'K39/25.1, K39/25.1, K39/25.2, K39/25.3.1, K39/25.3.1, K39/25.3.2, K39/25.3.3, K39/25.3.4, K39/25.3.6, K39/25.4, K39/25.6']
[2.0, 1, 'K87A/10.1, K87A/10.10, K87A/10.11, K87A/10.12, K87A/10.2,

[0.0, 5, 'L121/1, L121/58']
[0.0, 5, 'L121/60, L121/93']
[0.0, 5, 'L121/135, L121/82']
[0.0, 5, 'L121/112, L121/82']
[0.0, 5, 'L122/119, L122/76']
[0.0, 5, 'L122/102, L122/115']
[0.0, 5, 'L122/123, L122/139']
[0.0, 5, 'L122/128, L122/3']
[-1.0, 5, 'N481/33.1, negativecontrol1']
[-1.0, 5, 'N443/17.1, negativecontrol1']
[4.0, 5, 'D4/27.1, D4/49.1']
[-1.0, 5, 'K19/13.2, L120/48']
[4.0, 5, 'N440/13.1, N440/95.1']
[-1.0, 5, 'L119/17, N382/22.1']
[-1.0, 5, 'L119/17, N382/22.1']
[-1.0, 5, 'L119/17, N382/22.1']
[-1.0, 5, 'L119/17, N382/22.1']
[-1.0, 5, 'L119/17, N382/22.1']
[-1.0, 5, 'L119/17, N382/22.1']
[-1.0, 5, 'K42/21.12, L119/35']
[-1.0, 5, 'K42/21.12, L119/35']
[-1.0, 5, 'K42/21.12, L119/35']
[-1.0, 5, 'K42/21.12, L119/35']
[-1.0, 5, 'K42/21.12, L119/35']
[-1.0, 5, 'L119/41, N253/32.1']
[-1.0, 5, 'L119/41, N253/32.1']
[-1.0, 5, 'L119/47, L95A/44.1']
[-1.0, 5, 'L119/47, L95A/44.1']
[-1.0, 5, 'L119/47, L95A/44.1']
[-1.0, 5, 'L119/47, L95A/44.1']
[-1.0, 5, 'L119/87, L63/35.1']
[-1.0, 5, 'L

In [ ]:
# get categories that these fall into as well
mydictcount = {}
for index,row in data.iterrows():
    mab_id_list = [i.replace(' ','') for i in row['MabIDs'].split(',')]
    for mab_id in mab_id_list:
        if mab_id in mydictcount.keys():
            mydictcount[mab_id] +=1
        else:
            mydictcount[mab_id]=1

{k: v for k, v in sorted(mydictcount.items(), key=lambda item: item[1], reverse=True)}

In [28]:
# export this for belvin to check out
data.to_csv('updated_metadata.tsv',index=False, sep='\t')

# DROPOUTS

### lets see what the possible combinations possible are

In [32]:
set(dropout['Dropout? Missing'])

{'Dec2020dropout-Both', 'Dec2020dropout-HC', 'Dec2020dropout-LC', 'n/a-n/a'}

In [33]:
set(dropout['Dropout resolution'])

{'? Already redone',
 '? Already redone, non-IgG1/2a/2b/3?',
 '? Alternate subs also dropouts',
 '? Alternate subs do not corroborate each other',
 'Try again?',
 'n/a, Category 4 parent so skip for now',
 'n/a, empty',
 'n/a, got seq after all',
 'n/a, have good sequence for other subclone elsewhere',
 'n/a, have good sequence for same subclone elsewhere',
 'n/a, no show on web (control, expecting better seq from redo, just corroborating, etc.)',
 nan}

In [34]:
list(dropout['Dropout? Missing']).count('Dec2020dropout-Both')

793

### Lets groupby trimmer ID and then we can create some combinations of lists that are interesting.

In [35]:
grouped = dropout.groupby('trimmer_id')['Dropout? Missing'].apply(list).reset_index()

In [36]:
grouped['Dropout Resolution'] = dropout.groupby('trimmer_id')['Dropout resolution'].apply(list).reset_index()['Dropout resolution']

In [37]:
list_both = []
list_both_twice = []
list_HC = []
list_LC = []
list_HC_only = []
list_LC_only = []


for index, item in grouped.iterrows():

    if item['Dropout? Missing'].count("Dec2020dropout-Both")>=1 and item['Dropout? Missing'].count("Dec2020dropout-HC")>=1:
        list_HC.append(index)
    if item['Dropout? Missing'].count("Dec2020dropout-Both")>=1 and item['Dropout? Missing'].count("Dec2020dropout-LC")>=1:
        list_LC.append(index)
    if item['Dropout? Missing'].count("Dec2020dropout-Both")<1 and item['Dropout? Missing'].count("Dec2020dropout-HC")>=1:
        list_HC_only.append(index)
    if item['Dropout? Missing'].count("Dec2020dropout-Both")<1 and item['Dropout? Missing'].count("Dec2020dropout-LC")>=1:
        list_LC_only.append(index)
    if item['Dropout? Missing'].count("Dec2020dropout-Both")>=1:
        list_both.append(index)
    if item['Dropout? Missing'].count("Dec2020dropout-Both")>=2:
        list_both_twice.append(index)


In [45]:
grouped.ix[list_both].sort_values(by=['trimmer_id'])


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,trimmer_id,Dropout? Missing,Dropout Resolution
17,A12/18.1,"[Dec2020dropout-Both, n/a-n/a, n/a-n/a, n/a-n/a]","[n/a, have good sequence for same subclone els..."
20,D4/104.3,"[Dec2020dropout-Both, n/a-n/a]","[n/a, no show on web (control, expecting bette..."
24,D4/112.1,"[Dec2020dropout-Both, Dec2020dropout-HC]","[n/a, no show on web (control, expecting bette..."
27,D4/135.2,[Dec2020dropout-Both],[? Alternate subs also dropouts]
28,D4/135.3,[Dec2020dropout-Both],[? Alternate subs also dropouts]
...,...,...,...
6203,positivecontrol1_N18/22.1,[Dec2020dropout-Both],"[n/a, no show on web (control, expecting bette..."
6205,positivecontrol1_N263/31.1,"[Dec2020dropout-Both, Dec2020dropout-Both, Dec...","[n/a, no show on web (control, expecting bette..."
6208,positivecontrol1_N52A/42.1,"[Dec2020dropout-Both, n/a-n/a, n/a-n/a, n/a-n/...","[n/a, no show on web (control, expecting bette..."
6210,positivecontrol2,"[Dec2020dropout-Both, Dec2020dropout-Both, Dec...","[n/a, no show on web (control, expecting bette..."


In [46]:
grouped.sort_values(by=['trimmer_id']).to_csv('dropout_groupings.tsv', sep='\t')

In [40]:
# if there are other subclones from that we dont care about double dropouts

list([','.join(i) for i in grouped.ix[list_both_twice]['Dropout Resolution'] if type(i)==list and all(isinstance(i, str))])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


TypeError: 'bool' object is not iterable

In [ ]:
type('test')==str

In [ ]:
grouped.ix[list_HC]

In [ ]:
grouped.ix[list_LC]

In [ ]:
grouped.ix[list_HC_only]

In [ ]:
grouped.ix[list_LC_only]